In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install pyannote.audio
!pip install pydub

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [2]:
from google.colab import userdata
from huggingface_hub import login
import torch

# 1. Login to Hugging Face using the token stored in Colab Secrets
hf_token = userdata.get('dubbing_token')
login(token=hf_token)

# 2. Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wav_file = "/content/drive/MyDrive/dubbing/Article_38.wav"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# **Speaker Diarization (Who Spoke When?)**

In [1]:
from google.colab import userdata
from huggingface_hub import login
import torch
from pyannote.audio import Pipeline
from pydub import AudioSegment
import os

# 1. Login to Hugging Face using the token stored in Colab Secrets
hf_token = userdata.get('dubbing_token')
login(token=hf_token)

# 2. Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. Load the pre-trained pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1"
).to(device)

# 4. Convert MP3 to WAV first
# audio_file = "/content/drive/MyDrive/dubbing/input.mp3"
wav_file = "/content/drive/MyDrive/dubbing/Article_38.wav"

# # Convert MP3 to WAV at 16kHz mono
# audio = AudioSegment.from_mp3(audio_file)
# audio = audio.set_channels(1)  # Convert to mono
# audio = audio.set_frame_rate(16000)  # Set to 16kHz
# audio.export(wav_file, format="wav")

# Process the WAV file
diarization = pipeline(wav_file, min_speakers=1, max_speakers=5)

# 5. Create a structured list of segments
segments = []

# Access the actual Annotation object via the speaker_diarization attribute
annotation = diarization.speaker_diarization

# Now iterate over the annotation object
for segment, track, speaker in annotation.itertracks(yield_label=True):
    segment_data = {
        'start': segment.start,
        'end': segment.end,
        'speaker': speaker
    }
    segments.append(segment_data)
    print(f"Start={segment.start:.1f}s End={segment.end:.1f}s Speaker={speaker}")

# Now you have your segments list ready to use
print(f"\nTotal segments: {len(segments)}")

# Optional: Clean up the temporary WAV file
# os.remove(wav_file)

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

plda/xvec_transform.npz:   0%|          | 0.00/134k [00:00<?, ?B/s]

plda/plda.npz:   0%|          | 0.00/134k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pyannote/audio/models/blocks/pooling.py:103: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


Start=2.2s End=12.7s Speaker=SPEAKER_01
Start=13.1s End=15.4s Speaker=SPEAKER_00
Start=15.7s End=21.5s Speaker=SPEAKER_00
Start=22.1s End=24.1s Speaker=SPEAKER_01
Start=24.5s End=28.8s Speaker=SPEAKER_01
Start=31.2s End=34.5s Speaker=SPEAKER_00
Start=34.9s End=37.5s Speaker=SPEAKER_00
Start=37.7s End=39.7s Speaker=SPEAKER_00
Start=40.0s End=41.5s Speaker=SPEAKER_00
Start=41.9s End=51.9s Speaker=SPEAKER_01
Start=52.4s End=64.6s Speaker=SPEAKER_00
Start=64.7s End=86.5s Speaker=SPEAKER_00
Start=87.1s End=106.4s Speaker=SPEAKER_00
Start=106.9s End=107.5s Speaker=SPEAKER_01
Start=107.9s End=112.9s Speaker=SPEAKER_01
Start=113.9s End=115.6s Speaker=SPEAKER_00
Start=116.0s End=120.2s Speaker=SPEAKER_00
Start=121.2s End=122.8s Speaker=SPEAKER_01
Start=123.3s End=125.2s Speaker=SPEAKER_00
Start=125.2s End=137.1s Speaker=SPEAKER_00

Total segments: 20


In [2]:
import pandas as pd

# Convert list of dicts into DataFrame
df = pd.DataFrame(segments)

# Save as CSV
csv_file = "/content/drive/MyDrive/dubbing/Article_38_segments.csv"
df.to_csv(csv_file, index=False)

print(f"Segments saved to {csv_file}")

Segments saved to /content/drive/MyDrive/dubbing/Article_38_segments.csv


# **Transcription (What Was Said?)**

In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sixfdl63
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-sixfdl63
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
# Pseudocode/Example Logic
import whisper
from pydub import AudioSegment
import pandas as pd

# 1. Load the Whisper model
model = whisper.load_model("medium", device=device)

# 2. Load the full audio
full_audio = AudioSegment.from_mp3(wav_file)

# Load segments from CSV
df = pd.read_csv("/content/drive/MyDrive/dubbing/Article_38_segments.csv")

# Convert DataFrame back to list of dicts if needed
segments = df.to_dict(orient="records")

# print(f"Loaded {len(segments)} segments")


# 3. Transcribe each segment
for seg in segments:
    start_ms = int(seg['start'] * 1000)
    end_ms = int(seg['end'] * 1000)

    # Slice the audio
    audio_chunk = full_audio[start_ms:end_ms]

    # Whisper works with file paths, so save chunk temporarily
    chunk_path = "temp_chunk.wav"
    audio_chunk.export(chunk_path, format="wav")

    # Transcribe
    result = model.transcribe(chunk_path, fp16=torch.cuda.is_available())
    seg['text'] = result['text']

    print(f"[{seg['speaker']}]: {seg['text']}")

[SPEAKER_01]:  Ahmeed, your prices are driving all the small vendors like me out of business. It's not fair that you can just flood the market with lower prices, because you have more resources.
[SPEAKER_00]:  I'm just doing what's best for my business.
[SPEAKER_00]:  If I can offer lower prices, that's just competition. It's up to you to keep up or step aside.
[SPEAKER_01]:  But it's not just a bad competition.
[SPEAKER_01]:  It feels like you're using your advantage to push us out, and that's not right.
[SPEAKER_00]:  I see there's a dispute here regarding fairness in the marketplace.
[SPEAKER_00]:  Hi, I'm Raj, a government official.
[SPEAKER_00]:  Let's address this constructively.
[SPEAKER_00]:  what seems to be the problem.
[SPEAKER_01]:  Amit's larger shop has more resources, and he's using that to undercut prices and squeeze out smaller vendors like me. It's making it really hard for us to compete.
[SPEAKER_00]:  I'm just trying to offer better prices to attract customers. It's

In [12]:
import json

json_file = "/content/drive/MyDrive/dubbing/Article_38_segments_with_text.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(segments, f, indent=2, ensure_ascii=False)

print(f"Segments with text saved to {json_file}")

Segments with text saved to /content/drive/MyDrive/dubbing/Article_38_segments_with_text.json


# **Translation**

In [13]:
!pip install transformers sentencepiece

In [19]:
# Pseudocode/Example Logic
from transformers import pipeline

# 1. Define source, target, and load translation pipeline
source_lang = "en"  # English
target_lang = "hi"  # Spanish
model_name = f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}"
translator = pipeline("translation", model=model_name, device=0 if device=="cuda" else -1)

# 2. Translate text in each segment
for seg in segments:
    translated_result = translator(seg['text'])
    seg['translated_text'] = translated_result[0]['translation_text']
    print(f"Original: {seg['text']}")
    print(f"Translated: {seg['translated_text']}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Original:  Ahmeed, your prices are driving all the small vendors like me out of business. It's not fair that you can just flood the market with lower prices, because you have more resources.
Translated: यह उचित नहीं है कि आप बाजार कम कीमतों के साथ बाढ़ कर सकते हैं, क्योंकि आपके पास अधिक साधन हैं.
Original:  I'm just doing what's best for my business.
Translated: मैं सिर्फ अपने व्यापार के लिए सबसे अच्छा क्या कर रहा हूँ.
Original:  If I can offer lower prices, that's just competition. It's up to you to keep up or step aside.
Translated: अगर मैं कम कीमत दे सकते हैं, कि सिर्फ प्रतियोगिता है. यह आप के लिए ऊपर रखने या कदम रखने के लिए ऊपर या कदम रखने के लिए है.
Original:  But it's not just a bad competition.
Translated: लेकिन यह सिर्फ एक बुरी प्रतियोगिता नहीं है.
Original:  It feels like you're using your advantage to push us out, and that's not right.
Translated: ऐसा लगता है कि आप हमें धक्का देने के लिए अपने लाभ का उपयोग कर रहे हैं, और यह सही नहीं है.
Original:  I see there's a dispute here 

In [20]:
import json

json_file = "/content/drive/MyDrive/dubbing/Article_38_text_translated_hi.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(segments, f, indent=2, ensure_ascii=False)

print(f"Segments with text saved to {json_file}")

Segments with text saved to /content/drive/MyDrive/dubbing/Article_38_text_translated_hi.json


# **Voice Cloning and Assembling all clips**

**Refer [Voice cloning part](https://github.com/Somyajain2004/Auto-Dubbing-Project/blob/main/voice-cloning.ipynb)**